<a href="https://www.kaggle.com/code/tamalkoley/store-sales-prediction?scriptVersionId=162866057" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


Importing libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [3]:
train_path = "/kaggle/input/store-sales-time-series-forecasting/train.csv"
test_path = "/kaggle/input/store-sales-time-series-forecasting/test.csv"

In [4]:
train_df = pd.read_csv(train_path)
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
train_df = pd.get_dummies(train_df, columns = ['family'])
train_df.head()

,id,date,store_nbr,sales,onpromotion,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,0,2013-01-01,1,0.0,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2013-01-01,1,0.0,0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,2013-01-01,1,0.0,0,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,2013-01-01,1,0.0,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,4,2013-01-01,1,0.0,0,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [6]:
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['day_of_week'] = train_df['date'].dt.dayofweek
train_df['month'] = train_df['date'].dt.month
train_df[['id', 'date', 'day_of_week', 'month']]

,id,date,day_of_week,month
0,0,2013-01-01,1,1
1,1,2013-01-01,1,1
2,2,2013-01-01,1,1
3,3,2013-01-01,1,1
4,4,2013-01-01,1,1
...,...,...,...,...
3000883,3000883,2017-08-15,1,8
3000884,3000884,2017-08-15,1,8
3000885,3000885,2017-08-15,1,8
3000886,3000886,2017-08-15,1,8


Train the model with RandomForest

In [7]:
X = train_df.drop(['sales', 'date', 'onpromotion', 'store_nbr'], axis = 1)
y = train_df['sales']

Splitting data into training and testing sets

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Create and train the RandomForest model

In [9]:
model = RandomForestRegressor(n_estimators = 50, max_depth = 10, n_jobs = -1, random_state = 42)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=50, n_jobs=-1, random_state=42)

Prediction

In [10]:
y_pred = model.predict(X_test)

Root Mean Squared Logarithmic Error (RMSLE)

In [11]:
log_actual = np.log1p(y_test)
log_pred = np.log1p(y_pred)

rmsle = np.sqrt(np.mean(log_pred - log_actual) ** 2)
print("RMSLE:", rmsle)


RMSLE: 1.5047802674393285


Testing the data by applying the model for submission

In [12]:
test_df = pd.read_csv(test_path)
test_df = pd.get_dummies(test_df, columns = ['family'])

test_df['date'] = pd.to_datetime(test_df['date'])
test_df['day_of_week'] = test_df['date'].dt.dayofweek
test_df['month'] = test_df['date'].dt.month

X_submit = test_df.drop(['date', 'onpromotion', 'store_nbr'], axis = 1)
y_pred_submit = model.predict(X_submit)

submission_df = pd.DataFrame({'id': test_df['id'], 'sales': y_pred_submit})

submission_df.to_csv('submission.csv', index = False)